In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/financial-sheets-eda/final_finance_df.csv
/kaggle/input/financial-sheets-eda/__results__.html
/kaggle/input/financial-sheets-eda/__notebook__.ipynb
/kaggle/input/financial-sheets-eda/__output__.json
/kaggle/input/financial-sheets-eda/custom.css
/kaggle/input/financial-sheets/Readme.md
/kaggle/input/financial-sheets/dataset/dataset/Quarter_P_L_2_final.csv
/kaggle/input/financial-sheets/dataset/dataset/cash_flow_statments_final.csv
/kaggle/input/financial-sheets/dataset/dataset/t1_prices.csv
/kaggle/input/financial-sheets/dataset/dataset/Quarter_P_L_1_final.csv
/kaggle/input/financial-sheets/dataset/dataset/price_final.csv
/kaggle/input/financial-sheets/dataset/dataset/Balance_Sheet_final.csv
/kaggle/input/financial-sheets/dataset/dataset/Annual_P_L_2_final.csv
/kaggle/input/financial-sheets/dataset/dataset/ratios_1_final.csv
/kaggle/input/financial-sheets/dataset/dataset/ratios_2_final.csv
/kaggle/input/financial-sheets/dataset/dataset/other_metrics_final.csv
/kaggle/input

In [22]:
%%capture
# 1. Install Unsloth (Latest)
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# 2. Install dependencies WITHOUT the "<0.9.0" restriction on TRL
# We still use --no-deps on xformers to protect the PyTorch version
!pip install --no-deps xformers
!pip install --upgrade --no-cache-dir trl peft accelerate bitsandbytes datasets

In [23]:
price_path = "/kaggle/input/financial-sheets/dataset/dataset/price_final.csv"
metrics_path = "/kaggle/input/financial-sheets/dataset/dataset/other_metrics_final.csv"
balance_path = "/kaggle/input/financial-sheets/dataset/dataset/Balance_Sheet_final.csv"
pl_path = "/kaggle/input/financial-sheets/dataset/dataset/Annual_P_L_1_final.csv"

# Read CSVs
price_df = pd.read_csv(price_path)
metrics_df = pd.read_csv(metrics_path)
balance_df = pd.read_csv(balance_path)
pl_df = pd.read_csv(pl_path)

# Quick check
print("Price DataFrame shape:", price_df.shape)
print("Metrics DataFrame shape:", metrics_df.shape)
print("Balance Sheet shape:", balance_df.shape)
print("P&L DataFrame shape:", pl_df.shape)

# Optionally preview
price_df.head()

Price DataFrame shape: (4668, 34)
Metrics DataFrame shape: (4668, 44)
Balance Sheet shape: (4668, 51)
P&L DataFrame shape: (4668, 58)


,Name,BSE Code,NSE Code,Industry,Current Price,Return over 3months,Return over 6months,Volume 1month average,Volume 1week average,Volume,...,MACD Signal,MACD Signal Previous Day,Return over 1year,Return over 3years,Return over 5years,Volume 1year average,Return over 7years,Return over 10years,Market Capitalization,join_key
0,20 Microns,533022.0,20MICRONS,Mining / Minerals / Metals,225.20,44.62,23.16,729846,816052,312746,...,9.09,8.16,134.11,55.29,41.55,203559,28.58,20.15,794.66,533022.0_20MICRONS
1,21st Cent. Mgmt.,526921.0,21STCENMGM,Finance & Investments,70.44,54.12,114.43,34120,27069,11745,...,3.19,2.91,185.75,41.50,30.34,13416,15.32,24.91,73.96,526921.0_21STCENMGM
2,360 ONE,542772.0,360ONE,Finance & Investments,995.85,40.86,49.58,1193124,2871837,5329590,...,20.92,16.98,115.42,50.50,NaN,705325,NaN,NaN,36136.15,542772.0_360ONE
3,3B Blackbio,532067.0,__NA__,Healthcare,1184.50,48.07,25.94,21265,5355,7221,...,81.34,82.28,165.64,22.40,64.50,13985,54.44,64.15,1016.66,532067.0___NA__
4,3C IT Solutions,544190.0,__NA__,Computers - Software - Medium / Small,45.00,NaN,NaN,159500,81000,74000,...,0.72,0.82,NaN,NaN,NaN,159500,NaN,NaN,27.09,544190.0___NA__


In [24]:
price_cols = price_df.columns.tolist()
metrics_cols = metrics_df.columns.tolist()
balance_cols = balance_df.columns.tolist()
pl_cols = pl_df.columns.tolist()


# Print column names for each
print("📈 Price DataFrame columns:")
print(price_cols, "\n")

print("📊 Other Metrics DataFrame columns:")
print(metrics_cols, "\n")

print("📚 Balance Sheet DataFrame columns:")
print(balance_cols, "\n")

print("💰 Annual P&L DataFrame columns:")
print(pl_cols, "\n")

📈 Price DataFrame columns:
['Name', 'BSE Code', 'NSE Code', 'Industry', 'Current Price', 'Return over 3months', 'Return over 6months', 'Volume 1month average', 'Volume 1week average', 'Volume', 'High price', 'Low price', 'High price all time', 'Low price all time', 'Return over 1day', 'Return over 1week', 'Return over 1month', 'DMA 50', 'DMA 200', 'DMA 50 previous day', 'DMA 200 previous day', 'RSI', 'MACD', 'MACD Previous Day', 'MACD Signal', 'MACD Signal Previous Day', 'Return over 1year', 'Return over 3years', 'Return over 5years', 'Volume 1year average', 'Return over 7years', 'Return over 10years', 'Market Capitalization', 'join_key'] 

📊 Other Metrics DataFrame columns:
['Name', 'BSE Code', 'NSE Code', 'Industry', 'Current Price', 'Price to Sales', 'Price to Free Cash Flow', 'EVEBITDA', 'Current ratio', 'Interest Coverage Ratio', 'PEG Ratio', 'Working Capital to Sales ratio', 'QoQ Profits', 'QoQ Sales', 'Net worth', 'Market Cap to Sales', 'Interest Coverage', 'Enterprise Value to 

## Merging of Data for a Bigger Picture 

1.  **Price Data:** Real-time market performance (e.g., Current Price, RSI).
2.  **Valuation Metrics:** Derived ratios (e.g., P/E, PEG, Debt/Equity).
3.  **Balance Sheet:** Snapshot of financial position (Assets, Liabilities).
4.  **Annual P&L:** Historical operating performance (Sales, Expenses, Profits).

In [25]:
import pandas as pd
import numpy as np

# --- Helper Function for Export ---
def export_step(df, filename):
    """Saves a dataframe to CSV and prints a confirmation."""
    df.to_csv(filename, index=False)
    print(f"✅ Success: '{filename}' has been generated with {len(df)} rows.")

def merge_and_export_master(df_price, df_metrics, df_balance, df_pl, merge_key='BSE Code'):
    """
    Merges 4 dataframes into one Master Record and exports it.
    """
    print("🔄 Starting Merge Process...")
    
    # 1. Standardize Keys
    for df in [df_price, df_metrics, df_balance, df_pl]:
        df.columns = df.columns.str.strip() 
        if merge_key in df.columns:
            # Drop duplicates to prevent explosion
            df.drop_duplicates(subset=[merge_key], inplace=True)

    # 2. Merge Sequence (Outer Joins to preserve data)
    master = df_price.merge(df_metrics, on=merge_key, how='outer', suffixes=('', '_drop'))
    master = master.merge(df_balance, on=merge_key, how='outer', suffixes=('', '_drop'))
    master = master.merge(df_pl, on=merge_key, how='outer', suffixes=('', '_drop'))

    # 3. Clean Duplicate Columns
    cols_to_drop = [c for c in master.columns if '_drop' in c]
    master.drop(columns=cols_to_drop, inplace=True)
    
    # 4. EXPORT STEP 1: The Master File
    export_step(master, 'step1_master_financial_data.csv')
    
    return master


master_df = merge_and_export_master(price_df, metrics_df, balance_df, pl_df)

🔄 Starting Merge Process...
✅ Success: 'step1_master_financial_data.csv' has been generated with 4206 rows.


In [26]:
def transform_to_industry_formats(master_df):
    """
    Iterates through the Master DF, identifies the industry, 
    applies the specific format mapping, and saves 5 separate CSVs.
    """
    
    # Containers for our 5 formats
    saas_data = []
    retail_data = []
    mfg_data = []
    consulting_data = []
    hospitality_data = []
    
    # Iterate row by row
    for _, row in master_df.iterrows():
        industry = str(row.get('Industry', '')).lower()
        
        # --- COMMON CALCULATIONS ---
        sales = row.get('Sales', 0)
        op_profit = row.get('Operating profit', 0)
        net_profit = row.get('Net profit', 0)
        
        # Safe retrieval of costs (handling NaNs)
        material_cost = row.get('Material cost last year', 0) or 0
        employee_cost = row.get('Employee cost last year', 0) or 0
        depreciation = row.get('Depreciation', 0) or 0
        
        # --- MAPPING LOGIC ---
        
        # 1. SaaS / Tech Format
        if any(x in industry for x in ['software', 'it', 'tech', 'computers']):
            # Logic: Tech has low material cost. 'COGS' is usually server costs + support staff.
            # We assume 20% of Employee Cost is "Support" (COGS), rest is R&D/Admin.
            support_cost = employee_cost * 0.20
            gross_margin = sales - support_cost
            
            saas_data.append({
                'Company': row.get('Name'),
                'Recurring_Revenue': sales,
                'Cost_of_Revenue_Support': support_cost,
                'Gross_Profit': gross_margin,
                'OpEx_R_and_D_Staff': employee_cost * 0.80, # The rest of salaries
                'EBITDA': op_profit,
                'Net_Income': net_profit,
                'Metric_Price_to_Sales': row.get('Price to Sales')
            })

        # 2. Retail Format
        elif any(x in industry for x in ['retail', 'trading', 'department', 'fashion']):
            # Logic: Material Cost is the main COGS. Inventory is critical.
            retail_data.append({
                'Company': row.get('Name'),
                'Gross_Sales': sales,
                'COGS_Merchandise': material_cost,
                'Gross_Margin': sales - material_cost,
                'Store_Operating_Expenses': (sales - material_cost) - op_profit, # Inferred back-calc
                'Inventory_End_of_Period': row.get('Inventory'),
                'Working_Capital': row.get('Working capital'),
                'Net_Profit': net_profit
            })

        # 3. Manufacturing Format
        elif any(x in industry for x in ['steel', 'auto', 'chemical', 'pharma', 'textile']):
            # Logic: Heavy reliance on "Gross Block" (Machinery) and Depreciation.
            mfg_data.append({
                'Company': row.get('Name'),
                'Factory_Revenue': sales,
                'Direct_Materials': material_cost,
                'Factory_Overhead_Depreciation': depreciation,
                'Gross_Manufacturing_Profit': sales - material_cost - depreciation,
                'Plant_Machinery_Asset_Value': row.get('Gross block'),
                'Debt_Level': row.get('Debt'),
                'Net_Income': net_profit
            })

        # 4. Hospitality Format (Restaurants/Hotels)
        elif any(x in industry for x in ['hotel', 'restaurant', 'resort', 'hospitality']):
            # Logic: "Prime Cost" = Food Cost (Material) + Labor (Employee).
            prime_cost = material_cost + employee_cost
            hospitality_data.append({
                'Company': row.get('Name'),
                'Total_Revenue': sales,
                'COGS_Food_Beverage': material_cost,
                'Labor_Cost': employee_cost,
                'Prime_Cost': prime_cost, # The golden metric for this industry
                'Prime_Cost_Percentage': (prime_cost / sales) if sales > 0 else 0,
                'Rent_Occupancy_Cost': (sales - prime_cost - op_profit), # Inferred
                'EBITDA': op_profit
            })

        # 5. Consulting / Services (Default for others)
        else:
            # Logic: People business. No material cost. Revenue is fees.
            consulting_data.append({
                'Company': row.get('Name'),
                'Professional_Fees_Billed': sales,
                'Direct_Consultant_Salaries': employee_cost,
                'Contribution_Margin': sales - employee_cost,
                'Corporate_Overheads': (sales - employee_cost) - op_profit,
                'Net_Income': net_profit,
                'ROI_Metric': row.get('Return on capital employed')
            })

    # --- EXPORT STEP 2: The 5 Formats ---
    
    # Convert lists to DataFrames and Export
    outputs = [
        (pd.DataFrame(saas_data), 'step2_format_saas.csv'),
        (pd.DataFrame(retail_data), 'step2_format_retail.csv'),
        (pd.DataFrame(mfg_data), 'step2_format_manufacturing.csv'),
        (pd.DataFrame(consulting_data), 'step2_format_consulting.csv'),
        (pd.DataFrame(hospitality_data), 'step2_format_hospitality.csv')
    ]
    
    print("\n🔄 Generating Industry Formats...")
    for df, name in outputs:
        if not df.empty:
            export_step(df, name)
        else:
            print(f"⚠️ Warning: No data found for {name} (Check industry tags)")


transform_to_industry_formats(master_df)


🔄 Generating Industry Formats...
✅ Success: 'step2_format_saas.csv' has been generated with 349 rows.
✅ Success: 'step2_format_retail.csv' has been generated with 478 rows.
✅ Success: 'step2_format_manufacturing.csv' has been generated with 828 rows.
✅ Success: 'step2_format_consulting.csv' has been generated with 2481 rows.
✅ Success: 'step2_format_hospitality.csv' has been generated with 70 rows.


In [27]:
import pandas as pd
import datetime
import os
import re

# ==========================================
# 1. The Rendering Engine (Class)
# ==========================================
class IncomeStatementGenerator:
    def __init__(self):
        # Configuration: Maps Visual Labels to DataFrame Columns
        self.templates = {
            'SaaS': {
                'title': 'SaaS Income Statement',
                'schema': [
                    ('Recurring Revenue', 'Recurring_Revenue', False),
                    ('Cost of Revenue (Support)', 'Cost_of_Revenue_Support', True),
                    ('---', '---', False),
                    ('Gross Profit', 'Gross_Profit', False),
                    ('Operating Expenses (R&D)', 'OpEx_R_and_D_Staff', True),
                    ('---', '---', False),
                    ('EBITDA', 'EBITDA', False),
                    ('Net Income', 'Net_Income', False)
                ]
            },
            'Retail': {
                'title': 'Retail Operations P&L',
                'schema': [
                    ('Gross Sales', 'Gross_Sales', False),
                    ('COGS (Merchandise)', 'COGS_Merchandise', True),
                    ('---', '---', False),
                    ('Gross Margin', 'Gross_Margin', False),
                    ('Store OpEx', 'Store_Operating_Expenses', True),
                    ('---', '---', False),
                    ('Net Profit', 'Net_Profit', False),
                    ('===', '===', False),
                    ('Memo: Inventory', 'Inventory_End_of_Period', False)
                ]
            },
            'Manufacturing': {
                'title': 'Manufacturing Cost Sheet',
                'schema': [
                    ('Factory Revenue', 'Factory_Revenue', False),
                    ('Direct Materials', 'Direct_Materials', True),
                    ('Depreciation', 'Factory_Overhead_Depreciation', True),
                    ('---', '---', False),
                    ('Gross Mfg Profit', 'Gross_Manufacturing_Profit', False),
                    ('---', '---', False),
                    ('Net Income', 'Net_Income', False),
                    ('===', '===', False),
                    ('Memo: Plant Assets', 'Plant_Machinery_Asset_Value', False)
                ]
            },
            'Hospitality': {
                'title': 'Restaurant Statement of Operations',
                'schema': [
                    ('Total Revenue', 'Total_Revenue', False),
                    ('COGS (Food & Bev)', 'COGS_Food_Beverage', True),
                    ('Labor Cost', 'Labor_Cost', True),
                    ('---', '---', False),
                    ('PRIME COST', 'Prime_Cost', False),
                    ('Occupancy/Rent', 'Rent_Occupancy_Cost', True),
                    ('---', '---', False),
                    ('EBITDA', 'EBITDA', False)
                ]
            },
            'Consulting': {
                'title': 'Professional Services P&L',
                'schema': [
                    ('Professional Fees', 'Professional_Fees_Billed', False),
                    ('Direct Consultant Salaries', 'Direct_Consultant_Salaries', True),
                    ('---', '---', False),
                    ('Contribution Margin', 'Contribution_Margin', False),
                    ('Corp Overheads', 'Corporate_Overheads', True),
                    ('---', '---', False),
                    ('Net Income', 'Net_Income', False)
                ]
            }
        }

    def _format_currency(self, amount):
        if pd.isna(amount): return "$0.00"
        try:
            val = float(amount)
            # Use parentheses for negative accounting format
            if val < 0:
                return f"(${abs(val):,.2f})"
            return f"${val:,.2f}"
        except:
            return str(amount)

    def generate(self, company_name, format_type, data_row):
        template = self.templates.get(format_type)
        if not template:
            return f"Error: Template for '{format_type}' not found."

        # Header
        lines = []
        lines.append("="*60)
        lines.append(f"{str(company_name).upper()}")
        lines.append(f"{template['title']}")
        lines.append(f"Report Date: {datetime.date.today()}")
        lines.append("="*60)
        lines.append(f"{'Line Item':<40} | {'Amount':>15}")
        lines.append("-" * 60)

        # Body
        for label, col_key, is_expense in template['schema']:
            if label == '---':
                lines.append("-" * 60)
                continue
            if label == '===':
                lines.append("=" * 60)
                continue

            raw_val = data_row.get(col_key, 0)
            
            # Logic: If it's an expense line item, visually treat it as a deduction
            # Note: The raw data might already be positive. We display expenses in ()
            final_val = raw_val
            if is_expense and raw_val > 0:
                final_val = -raw_val
            
            val_str = self._format_currency(final_val)
            lines.append(f"{label:<40} | {val_str:>15}")

        lines.append("="*60 + "\n")
        return "\n".join(lines)


# ==========================================
# 2. The Batch Processor
# ==========================================
def run_batch_generation(dfs_dictionary, output_folder='Financial_Reports'):
    """
    Iterates through specific_dfs, generates P&Ls, and saves text files.
    """
    gen = IncomeStatementGenerator()
    
    # Create output directory
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"📁 Created directory: {output_folder}")
    
    report_count = 0
    
    print(f"🚀 Starting Batch Processing for {len(dfs_dictionary)} Industries...")
    
    for format_name, df in dfs_dictionary.items():
        if df.empty:
            print(f"⚠️  Skipping {format_name} (DataFrame is empty)")
            continue
            
        print(f"   ... Processing {format_name} ({len(df)} companies)")
        
        # Iterate through every company in this industry dataframe
        for idx, row in df.iterrows():
            company = row.get('Company', f'Unknown_Co_{idx}')
            
            # Generate the text content
            statement_text = gen.generate(
                company_name=company,
                format_type=format_name,
                data_row=row
            )
            
            # Sanitize filename (remove special chars from company name)
            safe_name = re.sub(r'[^\w\s-]', '', str(company)).strip().replace(' ', '_')
            filename = f"{output_folder}/{format_name}_{safe_name}.txt"
            
            # Save to file
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(statement_text)
            
            report_count += 1

    print(f"\n✅ Batch Complete! Generated {report_count} reports in '/{output_folder}'")


# ==========================================
# 3. RUN IT (Using your specific_dfs)
# ==========================================

# Assuming 'specific_dfs' is already loaded in your environment from the previous steps.
# If not, uncomment the lines below to load them from the CSVs we made earlier:

specific_dfs = {
    'SaaS': pd.read_csv('step2_format_saas.csv'),
    'Retail': pd.read_csv('step2_format_retail.csv'),
    'Manufacturing': pd.read_csv('step2_format_manufacturing.csv'),
    'Hospitality': pd.read_csv('step2_format_hospitality.csv'),
    'Consulting': pd.read_csv('step2_format_consulting.csv')
}

# Execute
if 'specific_dfs' in locals():
    run_batch_generation(specific_dfs)
else:
    print("❌ Error: 'specific_dfs' variable not found. Please ensure CSVs are loaded.")

🚀 Starting Batch Processing for 5 Industries...
   ... Processing SaaS (349 companies)
   ... Processing Retail (478 companies)
   ... Processing Manufacturing (828 companies)
   ... Processing Hospitality (70 companies)
   ... Processing Consulting (2481 companies)

✅ Batch Complete! Generated 4206 reports in '/Financial_Reports'


In [28]:
import torch
from unsloth import FastLanguageModel

major_version, minor_version = torch.cuda.get_device_capability()
print(f"✅ GPU Detected: CUDA Capability {major_version}.{minor_version}")
print("✅ Unsloth installed successfully.")

✅ GPU Detected: CUDA Capability 7.5
✅ Unsloth installed successfully.


In [29]:
import pandas as pd
import json
import os

def convert_to_unsloth_jsonl(file_map, output_filename="financial_finetune.jsonl"):
    """
    Reads multiple CSVs and converts them into a single JSONL file 
    formatted for Unsloth / Alpaca fine-tuning.
    
    Args:
        file_map (dict): Keys are 'Industry Name', Values are 'path_to_csv'.
        output_filename (str): The name of the output JSONL file.
    """
    
    combined_data = []
    
    print(f"🔄 Starting conversion to {output_filename}...")
    
    for industry, filepath in file_map.items():
        if not os.path.exists(filepath):
            print(f"⚠️  Skipping {industry}: File not found ({filepath})")
            continue
            
        df = pd.read_csv(filepath)
        print(f"   ... Processing {industry} ({len(df)} rows)")
        
        for _, row in df.iterrows():
            # 1. Clean the data (Handle NaNs)
            # We convert row to dict and replace NaN with None or 0
            row_dict = row.where(pd.notnull(row), None).to_dict()
            
            # 2. Construct the Prompt components
            company_name = row_dict.get('Company', 'a company')
            
            # INSTRUCTION: The prompt you will give the AI
            # We vary it slightly to make the model robust
            instruction = f"Generate a {industry} Income Statement for {company_name}."
            
            # INPUT: Optional context (usually empty for pure generation tasks)
            # But we can add context if we want to simulate 'formatting' raw data
            input_context = "" 
            
            # OUTPUT: The exact JSON you want the AI to learn to generate
            # We dump the row_dict as a JSON string
            output_json = json.dumps(row_dict)
            
            # 3. Create the Training Entry
            entry = {
                "instruction": instruction,
                "input": input_context,
                "output": output_json
            }
            
            combined_data.append(entry)

    # 4. Save to JSONL
    with open(output_filename, 'w') as f:
        for entry in combined_data:
            f.write(json.dumps(entry) + '\n')
            
    print(f"✅ Success! Created {output_filename} with {len(combined_data)} training examples.")
    print("   You can now load this into Unsloth using load_dataset('json', data_files='...')")

# ==========================================
# EXECUTION
# ==========================================

# Map your CSVs from the previous step
csv_files = {
    'SaaS': 'step2_format_saas.csv',
    'Retail': 'step2_format_retail.csv',
    'Manufacturing': 'step2_format_manufacturing.csv',
    'Hospitality': 'step2_format_hospitality.csv',
    'Consulting': 'step2_format_consulting.csv'
}

# Run the converter
convert_to_unsloth_jsonl(csv_files)

🔄 Starting conversion to financial_finetune.jsonl...
   ... Processing SaaS (349 rows)
   ... Processing Retail (478 rows)
   ... Processing Manufacturing (828 rows)
   ... Processing Hospitality (70 rows)
   ... Processing Consulting (2481 rows)
✅ Success! Created financial_finetune.jsonl with 4206 training examples.
   You can now load this into Unsloth using load_dataset('json', data_files='...')


In [30]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

# 1. Configuration
max_seq_length = 2048
dtype = None # Auto detection
load_in_4bit = True # Use 4bit quantization to reduce memory usage

# 2. Load Base Model (Llama-3 8B is excellent for JSON structure)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. Add LoRA Adapters (The "Fine-Tuning" Magic)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# 4. Prompt Formatting
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# 5. Load Your Dataset
# Replace 'your_dataset.jsonl' with the file from Step 1
dataset = load_dataset("json", data_files="financial_finetune.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

# 6. Train the Model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        
        # --- HARDWARE FIXES FOR P100 ---
        fp16 = True,           # P100 requires fp16
        bf16 = False,          # P100 cannot do bf16
        optim = "adamw_8bit",
        
        # --- PREVENT HANGING ---
        report_to = "none",    # Disables WandB hanging
        logging_steps = 1,     # Print progress every step
        
        output_dir = "outputs",
        seed = 3407,
    ),
)

trainer.train()
print("✅ Training Complete! Model is now a Financial Expert.")

==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4206 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/4206 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 4,206 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,2.687200
2,2.680200
3,2.659500
4,2.768000
5,2.614800
6,2.687800
7,2.617400
8,2.450900
9,2.403200
10,2.212600


✅ Training Complete! Model is now a Financial Expert.


In [34]:
import torch
from unsloth import FastLanguageModel
import json
import plotly.graph_objects as go
import re

# ==========================================
# 1. SETUP & CONFIGURATION
# ==========================================
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Define the Prompt Template (Must match training data EXACTLY)
# Note: We removed the '### Input:' section to prevent model confusion
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
"""

# The specific request
instruction = "Generate a SaaS Income Statement for a high-growth AI startup."

# ==========================================
# 2. RUN INFERENCE (GENERATION)
# ==========================================
print("🤖 Generating Synthetic Financials...")
inputs = tokenizer(
    [alpaca_prompt.format(instruction)], 
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    **inputs, 
    max_new_tokens=512, 
    use_cache=True,
    temperature=0.8, # Higher temp = more creative numbers
)

# Decode the output
generated_text = tokenizer.batch_decode(outputs)[0]

# ==========================================
# 3. PARSE JSON (ROBUSTLY)
# ==========================================
financial_data = None
try:
    # 1. Isolate the JSON part (everything after "### Response:")
    response_part = generated_text.split("### Response:")[-1].strip()
    response_part = response_part.replace("<|end_of_text|>", "")
    
    # 2. Attempt to parse
    financial_data = json.loads(response_part)
    print("\n✅ Successfully Parsed JSON:")
    print(json.dumps(financial_data, indent=2))

except json.JSONDecodeError:
    # Fallback: Try to find the first '{' and last '}' using Regex
    try:
        match = re.search(r"\{.*\}", response_part, re.DOTALL)
        if match:
            financial_data = json.loads(match.group(0))
            print("\n✅ Recovered JSON via Regex:")
            print(json.dumps(financial_data, indent=2))
        else:
            print("\n❌ Parsing Failed. Raw Output below:")
            print(response_part)
    except:
        print("\n❌ Critical Parsing Error. Raw Output below:")
        print(response_part)

# ==========================================
# 4. VISUALIZE (WATERFALL CHART)
# ==========================================
def plot_robust_waterfall(data):
    if not data: return

    # --- Helper to handle AI Aliases ---
    def get_val(keys):
        for k in keys:
            if k in data: return data[k]
        return 0

    # 1. Extract Values (Looking for variations in keys)
    revenue = get_val(['Recurring_Revenue', 'Revenue', 'Total_Revenue', 'Sales', 'Gross_Sales'])
    cor     = get_val(['Cost_of_Revenue', 'Cost_of_Revenue_Support', 'COGS', 'Direct_Costs'])
    gp      = get_val(['Gross_Profit', 'Gross_Margin'])
    
    # If Gross Profit is 0/Missing, calculate it
    if gp == 0 and revenue > 0:
        gp = revenue - cor

    # Operating Expenses
    rnd     = get_val(['R_and_D', 'OpEx_R_and_D_Staff', 'Research_and_Development'])
    sales   = get_val(['Sales_Marketing', 'S_and_M', 'Marketing'])
    ga      = get_val(['G_and_A', 'General_Administrative', 'Admin'])
    
    # If granular OpEx is missing, look for a "Total OpEx" key
    total_opex = rnd + sales + ga
    if total_opex == 0:
        total_opex = get_val(['Operating_Expenses', 'OpEx', 'Total_Expenses'])

    # EBITDA
    ebitda = get_val(['EBITDA', 'Operating_Income', 'Operating_Profit'])
    if ebitda == 0:
        ebitda = gp - total_opex

    # 2. Build Plot Data
    fig = go.Figure(go.Waterfall(
        name = "2025", orientation = "v",
        measure = ["absolute", "relative", "total", "relative", "relative", "relative", "total"],
        x = ["Revenue", "Cost of Revenue", "Gross Profit", "R&D", "Sales & Mkt", "G&A", "EBITDA"],
        textposition = "outside",
        text = [f"${x/1e6:.1f}M" if x!=0 else "" for x in [revenue, -cor, gp, -rnd, -sales, -ga, ebitda]],
        y = [revenue, -cor, 0, -rnd, -sales, -ga, 0],
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
        decreasing = {"marker":{"color":"#EF553B"}},
        increasing = {"marker":{"color":"#00CC96"}},
        totals     = {"marker":{"color":"#19D3F3"}}
    ))

    fig.update_layout(
        title = f"Synthetic P&L: {data.get('Company', 'AI Startup')}",
        template = "plotly_dark",
        showlegend = False,
        height=650
    )
    
    fig.show()

# Run Plot
if financial_data:
    plot_robust_waterfall(financial_data)

🤖 Generating Synthetic Financials...

✅ Successfully Parsed JSON:
{
  "Company": "Mantle AI",
  "Recurring_Revenue": 6.7,
  "Contribution_Margin": 3.7,
  "Corporate_Overheads": 3.3,
  "Net_Income": 0.6,
  "Weighted_Average_Shares_Outstanding": 8.1,
  "Revenue_Growth": 1.3,
  "Contribution_Margin_Growth": 1.3,
  "Net_Income_Growth": 1.3,
  "Free_Cash_Flow": 1.2,
  "Adjusted_Free_Cash_Flow": 1.2,
  "Gross_Margin": 5.4
}
